In [1]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib as mpl
from random import random
from random import sample
from statsmodels.sandbox.distributions.extras import pdf_mvsk
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import scipy.stats as stats
import datetime
from copy import deepcopy
from time import time

In [102]:
class Game:
    def __init__(self, home_team, away_team):
        self.home = home_team
        self.away = away_team
        self.plays = {}

    def is_same_game(self, row):
        if row['home'] == self.home and row['away'] == self.away:
            return True
        return False

    def process_row(self, row, next_row=None, last_row=None):
        def add_half_details():
            if row['half'] == 1:
                play['half'] = 1
                play['overtime'] = 0
            elif row['half'] == 2:
                play['half'] = 2
                play['overtime'] = 0
            elif row['half'] == 'Overtime':
                play['half'] = None
                play['overtime'] = 1
            else:
                print('ERROR #0 - row[\'half\'] != \'1\' or \'2\' or \'Overtime\'')
                play['half'] = None

        def add_pass_details():
            rush_plays = ['Rush', 'Rushing Touchdown']
            pass_plays = ['Sack', 'Pass Reception', 'Passing Touchdown',
                               'Pass Incompletion', 'Pass', 'Pass Completion']
            # field_goal_plays = ['Field Goal Good', 'Field Goal Missed', 'Blocked Field Goal']
            turnover_plays = ['Fumble Recovery (Own)', 'Fumble Recovery (Opponent)',
                              'Interception Return Touchdown', 'Fumble Recovery (Opponent) Touchdown',
                              'Interception Return']
            if row['play_type'] in rush_plays:
                play['run_vs_pass'] = 1
            elif row['play_type'] in pass_plays:
                play['run_vs_pass'] = 0
                play['pass_complete'] = 1 if row['completion'] == 1 else 0
            elif row['play_type'] in turnover_plays:
                if not pd.isna(row['rusher_player_name']):
                    play['run_vs_pass'] = 1
                elif not pd.isna(row['passer_player_name']) or pd.isna(row['receiver_player_name']):
                    play['run_vs_pass'] = 0
                    play['is_sack_fumble'] = 1
                # play['run_vs_pass'] = None
            else:
                play['run_vs_pass'] = None
            #     play['deep_pass'] = None
            #     play['run_vs_pass'] = 1
            # elif row['pass_length'] == 'short':
            #     play['run'] = 0
            #     play['short_pass'] = 1
            #     play['deep_pass'] = 0
            #     play['run_vs_pass'] = 0
            # elif row['pass_length'] == 'deep':
            #     play['run'] = 0
            #     play['short_pass'] = 0
            #     play['deep_pass'] = 1
            #     play['run_vs_pass'] = 0
            # else:
            #     # this is reached if its a field goal or so
            #     play['run'] = 0
            #     play['short_pass'] = 0
            #     play['deep_pass'] = 0
            #     play['run_vs_pass'] = None

            # if row['sack'] == 1:
            #     play['run_vs_pass'] = 0


        def add_type_of_play_details():
            # if pd.isna(row['play_type']):
                # play['type'] = 'unknown'
            # elif 'TWO-POINT CONVERSION' in row['desc']:
            #     play['type'] = '2pt'
            #     if row['two_point_conv_result'] == 'success':
            #         play['2pt_success'] = 1
            #     elif row['two_point_conv_result'] == 'failure':
            #         play['2pt_success'] = 0
            # elif 'extra_point' in row['play_type']:
            #     play['type'] = '1pt'
            #     if row['extra_point_result'] == 'good':
            #         play['1pt_success'] = 1
            #     else:
            #         play['1pt_success'] = 0
            # else:
            play['type'] = row['play_type']

        # def add_home_away_score_details():
        #     play['home_score'] = row['total_home_score']
        #     play['away_score'] = row['total_away_score']
        #     if play['home_score'] > play['away_score']:
        #         play['home_winning'] = 1
        #         play['away_winning'] = 0
        #         play['scoreless_game'] = 0
        #         play['tie'] = 0
        #     elif play['away_score'] > play['home_score']:
        #         play['home_winning'] = 0
        #         play['away_winning'] = 1
        #         play['scoreless_game'] = 0
        #         play['tie'] = 0
        #     elif play['away_score'] == 0:
        #         play['home_winning'] = None
        #         play['away_winning'] = None
        #         play['scoreless_game'] = 1
        #         play['tie'] = 1
        #     else:
        #         play['home_winning'] = None
        #         play['away_winning'] = None
        #         play['scoreless_game'] = 0
        #         play['tie'] = 1

        def add_pos_def_score_details():
            play['pos_score'] = row['pos_team_score']
            play['def_score'] = row['def_pos_team_score']
            play['pos_minus_def_score'] = play['pos_score'] - play['def_score']
            if play['pos_score'] > play['def_score']:
                play['pos_winning'] = 1
                play['def_winning'] = 0
                play['tie'] = 0
            elif play['def_score'] > play['pos_score']:
                play['pos_winning'] = 0
                play['def_winning'] = 1
                play['tie'] = 0
            else:
                play['pos_winning'] = 0
                play['def_winning'] = 0
                play['tie'] = 1

        # def add_down_converted():
        #     if row['ydstogo'] <= row['yards_gained']:
        #         play['converted'] = True
        #     else:
        #         play['converted'] = False

        # def add_td():
        #     if row['posteam_score_post'] == row['posteam_score'] + 6:
        #         play['pos_td'] = 1
        #     else:
        #         play['pos_td'] = 0
        #     if row['defteam_score_post'] == row['defteam_score'] + 6:
        #         play['def_td'] = 1
        #     else:
        #         play['def_td'] = 0

        def add_time_of_play():
            if last_row is None:
                play['play_t_length'] = 0#row['TimeSecsRem']
                # play['last_play_of_game'] = 1
            else:
                play['play_t_length'] = last_row['TimeSecsRem'] - row['TimeSecsRem']
                play['last_play_of_game'] = 0
                if play['play_t_length'] > 65 or play['play_t_length'] < 1:
                    play['play_t_length'] = None

        # def add_penalty():
        #     if row['penalty']:
        #         play['penalty'] = 1
        #         play['penalty_1st'] = row['first_down_penalty']
        #         play['penalty_team'] = row['penalty_team']
        #         if play['penalty_team'] == play['pos_team']:
        #             play['penalty_pos'] = 1
        #         else:
        #             play['penalty_pos'] = 0
        #         play['penalty_type'] = row['penalty_type']
        #         play['penalty_yd'] = row['penalty_yards']
        #         if 'declined' in row['desc']:
        #             play['penalty_acpt'] = 0
        #         else:
        #             play['penalty_acpt'] = 1
        #     else:
        #         play['penalty'] = 0

        def add_turnover():
            play['fumble'] = row['fumble_vec']
            #play['fumble_out'] = row['fumble_out_of_bounds']
            if play['fumble']:
                play['fumble_lost'] = row['turnover_vec']
                play['fumble_spot_yds'] = row['yards_gained']
                if play['fumble_lost']:
                    play['fumble_spot_yds'] -= row['yds_fumble_return']
                else:
                    play['fumble_spot_yds'] += row['yds_fumble_return']
            else:
                set_none(['fumble_lost', 'fumble_spot_yds'])

            play['interception'] = 1 if row['interception_stat']==1 else 0
            if play['interception']:
                play['int_spot_yds'] = row['yds_int_return']
            else:
                play['int_spot_yds'] = None

        def add_punt():
            play['punt'] = row['punt']
            if play['punt']==1:
                play['punt_blocked'] = row['punt_blocked']
                if row['punt_tb']==1:
                    play['punt_yds'] = play['yards_to_goal'] - 20
                else:
                    play['punt_yds'] = row['yds_punted'] - row['yds_punt_return']
            else:
                set_none(['punt_blocked', 'punt_yds'])

        def add_FG():
            play['pos_down_three_or_less'] = 1 if -3 <= play['pos_minus_def_score'] <= 0 else 0
            if not pd.isna(row['fg_inds']):
                play['FG_attempted'] = int(row['fg_inds'])
                play['FG_made'] = 1 if row['fg_made'] else 0
            else:
                play['FG_attempted'] = 0
                play['FG_made'] = None
            # if play['FG_attempted']==1:
                # play['FG_made'] = row['fg_made']
            # else:
                # play['FG_made'] = None
                  
        def set_none(list_of_keys):
            for key in list_of_keys:
                play[key] = None

        # def add_week():
        #     play['year'] = row['game_date'].year
        #     play['month'] = row['game_date'].month
        #     play['day'] = row['game_date'].day
        #     if play['day'] < 10:
        #         week_str = str(play['month']) + '0' + str(play['day'])
        #     else:
        #         week_str = str(play['month']) + str(play['day'])
        #     play['week'] = DATE_NUMBER_TO_WEEK_NUMBER[play['year']][week_str]

        play = {}
        play['game_id'] = row['game_id']
        play['time_left'] = row['adj_TimeSecsRem']
        play['half_left'] = row['TimeSecsRem']
        play['qt_left'] = row['TimeSecsRem']%900 if row['TimeSecsRem']%900 != 0 else 900

        play['home'] = self.home
        play['away'] = self.away
        play['down'] = row['down']
        play['yards_gained'] = row['yards_gained']
        play['distance'] = row['distance']
        # play['ydsnet'] = row['ydsnet']
        
        play['id'] = 'g' +  str(row['game_id']) + 'p' + str(row['id_play'])
        
            
        play['pos_team'] = row['pos_team']
        play['def_team'] = row['def_pos_team']
        play['yards_to_goal'] = row['yards_to_goal']
        play['qtr'] = row['period']
        add_half_details()
        add_type_of_play_details()
        add_pass_details()
        # add_home_away_score_details()
        add_pos_def_score_details()
        # add_down_converted()
        # add_td()
        add_time_of_play()
        # add_penalty()
        add_turnover()
        add_punt()
        add_FG()
        # add_week()
        # Check if play with the same ID already exists in the dictionary
        
        # handle dupes
        if play['id'] in self.plays:
            count = 1
            new_play_id = f"{play['id']}_{count}"
            while new_play_id in self.plays:
                count += 1
                new_play_id = f"{play['id']}_{count}"
            play['id'] = new_play_id
        
        self.plays[play['id']] = play

def iterate_df(df):
    print('Running iterate_df...')
    print('Number of rows:', df.shape[0])
    games = []
    plays = {}
    i = 0
    df.index = np.arange(0, len(df)) # index will start at some high number if not year 2009, the first year of data xl
    for idx, row in tqdm(df.iterrows()):
        if not i:
            game = Game(row['home'], row['away'])
            i += 1
        if not game.is_same_game(row):
            games.append(game)
            game = Game(row['home'], row['away'])

        elif idx + 1 < df.shape[0]:
            game.process_row(row, next_row=df.iloc[idx+1], last_row=df.iloc[idx-1])
        else:
            game.process_row(row, last_row=df.iloc[idx-1])
        
        # Check and update only new plays
        # play_id = row['id']
        # play_id = 'g' +  str(row['game_id']) + 'p' + str(row['id_play'])
        # if play_id not in plays:
        #     plays[play_id] = game.plays[play_id]
        
        plays.update(game.plays)
                
    plays_df = pd.DataFrame.from_dict(plays, 'index')
    
    return plays_df

In [66]:
sample_df.play_type.unique()

array(['Kickoff', 'Rush', 'Pass Incompletion', 'Punt', 'Pass Reception',
       'Sack', 'Penalty', 'Field Goal Good', 'Rushing Touchdown',
       'Interception Return', 'End Period', 'Timeout', 'End of Half',
       'Passing Touchdown', 'Fumble Recovery (Own)',
       'Kickoff Return (Offense)', 'Field Goal Missed',
       'Fumble Recovery (Opponent)', 'Blocked Punt Touchdown',
       'Punt Return Touchdown', 'Punt Team Fumble Recovery',
       'Blocked Field Goal', 'Safety', 'Interception Return Touchdown',
       'Fumble Recovery (Opponent) Touchdown',
       'Kickoff Team Fumble Recovery', 'Fumble Return Touchdown',
       'Kickoff Return Touchdown', 'Blocked Punt', 'End of Game',
       'Defensive 2pt Conversion', nan, 'Missed Field Goal Return',
       'Uncategorized', 'Punt (Safety)'], dtype=object)

In [4]:
df = pd.read_csv("pbpdata2018_2023.csv", encoding='ISO-8859-1')

c:\Users\punco\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (74,133,134,135,138,146,147,150,151,162,225,241,242,243,244,245,246,247,248,249,250,251,252,253,254,256,260,263,266,269,272,275,278,281,284,287,290,293,296,299) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [23]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

list(sample_df.columns)

['Unnamed: 0',
 'year',
 'week',
 'id_play',
 'game_id',
 'game_play_number',
 'half_play_number',
 'drive_play_number',
 'pos_team',
 'def_pos_team',
 'pos_team_score',
 'def_pos_team_score',
 'half',
 'period',
 'clock.minutes',
 'clock.seconds',
 'play_type',
 'play_text',
 'down',
 'distance',
 'yards_to_goal',
 'yards_gained',
 'EPA',
 'ep_before',
 'ep_after',
 'wpa',
 'wp_before',
 'wp_after',
 'def_wp_before',
 'def_wp_after',
 'penalty_detail',
 'yds_penalty',
 'penalty_1st_conv',
 'new_series',
 'firstD_by_kickoff',
 'firstD_by_poss',
 'firstD_by_penalty',
 'firstD_by_yards',
 'def_EPA',
 'home_EPA',
 'away_EPA',
 'home_EPA_rush',
 'away_EPA_rush',
 'home_EPA_pass',
 'away_EPA_pass',
 'total_home_EPA',
 'total_away_EPA',
 'total_home_EPA_rush',
 'total_away_EPA_rush',
 'total_home_EPA_pass',
 'total_away_EPA_pass',
 'net_home_EPA',
 'net_away_EPA',
 'net_home_EPA_rush',
 'net_away_EPA_rush',
 'net_home_EPA_pass',
 'net_away_EPA_pass',
 'success',
 'epa_success',
 'rz_play',
 

In [103]:
game_ids = list(df.game_id.unique())
game_ids_smp = sample(game_ids, 100)
sample_df = df[df['game_id'].isin(game_ids_smp)]

In [104]:
# Test here
if __name__ == '__main__':
    # df = pd.read_csv('pbpdata2018_2023.csv', encoding='ISO-8859-1')
    plays_df = iterate_df(sample_df)

0it [00:00, ?it/s]

Running iterate_df...
Number of rows: 17794


17794it [00:21, 839.57it/s] 


In [101]:
sample_df[sample_df['play_type'].isin(['Pass Incompletion', 'Pass Reception'])][['play_type', 'completion', 'incompletion_stat', 'completion_yds', 'yards_gained']]

,play_type,completion,incompletion_stat,completion_yds,yards_gained
3,Pass Incompletion,0,1.0,NaN,0
7,Pass Reception,1,NaN,5.0,5
10,Pass Reception,1,NaN,1.0,1
11,Pass Reception,1,NaN,7.0,7
15,Pass Reception,1,NaN,11.0,11
23,Pass Incompletion,0,1.0,NaN,0
26,Pass Reception,1,NaN,34.0,34
29,Pass Reception,1,NaN,3.0,3
35,Pass Reception,1,NaN,14.0,14
...,...,...,...,...,...


In [105]:
plays_df.head()

,game_id,time_left,half_left,qt_left,home,away,down,yards_gained,distance,id,pos_team,def_team,yards_to_goal,qtr,half,overtime,type,run_vs_pass,pos_score,def_score,pos_minus_def_score,pos_winning,def_winning,tie,play_t_length,last_play_of_game,fumble,fumble_lost,fumble_spot_yds,interception,int_spot_yds,punt,punt_blocked,punt_yds,pos_down_three_or_less,FG_attempted,FG_made,pass_complete,is_sack_fumble
g401013107p4.010131071018499e+17,401013107,3600,1800,900,Virginia Tech,William & Mary,1.0,27,10.0,g401013107p4.010131071018499e+17,Virginia Tech,William & Mary,65,1,1,0,Kickoff,NaN,0,0,0,0,0,1,NaN,0,0,NaN,NaN,0,NaN,0,NaN,NaN,1,0,0.0,NaN,NaN
g401013107p4.010131071018499e+17_1,401013107,3600,1800,900,Virginia Tech,William & Mary,1.0,18,10.0,g401013107p4.010131071018499e+17_1,Virginia Tech,William & Mary,72,1,1,0,Pass Reception,0.0,0,0,0,0,0,1,NaN,0,0,NaN,NaN,0,NaN,0,NaN,NaN,1,0,0.0,1.0,NaN
g401013107p4.010131071018499e+17_2,401013107,3600,1800,900,Virginia Tech,William & Mary,1.0,18,10.0,g401013107p4.010131071018499e+17_2,Virginia Tech,William & Mary,54,1,1,0,Rush,1.0,0,0,0,0,0,1,NaN,0,0,NaN,NaN,0,NaN,0,NaN,NaN,1,0,0.0,NaN,NaN
g401013107p4.010131071018499e+17_3,401013107,3600,1800,900,Virginia Tech,William & Mary,1.0,2,10.0,g401013107p4.010131071018499e+17_3,Virginia Tech,William & Mary,36,1,1,0,Fumble Recovery (Opponent),1.0,0,0,0,0,0,1,NaN,0,1,1.0,NaN,0,NaN,0,NaN,NaN,1,0,0.0,NaN,NaN
g401013107p4.0101310710186496e+17,401013107,3529,1729,829,Virginia Tech,William & Mary,1.0,-1,10.0,g401013107p4.0101310710186496e+17,William & Mary,Virginia Tech,66,1,1,0,Pass Reception,0.0,0,0,0,0,0,1,NaN,0,0,NaN,NaN,0,NaN,0,NaN,NaN,1,0,0.0,1.0,NaN


In [106]:
# Write to .csv for use in CFB Sim
plays_df.to_csv('sample_data.csv')